In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import clip.clip as clip 

CLIP_MODEL,  clip_img_processor =clip.load('ViT-B/32', jit=False)

In [3]:
from src.data_loader import read_data

test_data = read_data('test')

100%|██████████| 200/200 [00:00<00:00, 1496.53it/s]


In [4]:
from src.config import BATCH_SIZE
import random 
import numpy as np 
import torch 

from src.networks.CLIP_distilBERT import CLIP_dBERT_dataset

def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)
g = torch.Generator()
g.manual_seed(0)

def get_dataloader(data):
    dataset = CLIP_dBERT_dataset(data, clip_img_processor)
    data_loader = torch.utils.data.DataLoader(dataset, batch_size=BATCH_SIZE,shuffle=True, worker_init_fn=seed_worker, generator=g)
    return data_loader

test_loader = get_dataloader(test_data)

CLIP preprocessing: 100%|██████████| 200/200 [00:04<00:00, 44.38it/s]


In [5]:
from src.trainer import Trainer
from src.networks.CLIP_distilBERT import CLIP_dBERT_Input_transformer, CLIP_dBERT_Model
import torch.nn as nn
import torch 
STATE_DICT =  torch.load('model.torch')
model = CLIP_dBERT_Model(CLIP_MODEL)
model.load_state_dict(STATE_DICT)
model = model.cuda()

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_projector.bias', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [6]:
from src.networks.CLIP_distilBERT import CLIP_dBERT_Input_transformer
input_transformer = CLIP_dBERT_Input_transformer()

In [7]:
from captum.attr import IntegratedGradients
ig = IntegratedGradients(model)


In [8]:
x, y= next(iter(test_loader)) 
x,y = input_transformer.transform(x,y)

c:\Users\ADITYA RATHORE\Desktop\CV_Project\src\networks\CLIP_distilBERT.py:51: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  id = torch.tensor(b_input_ids).to(self.device).long()


In [9]:
score = ig.attribute(x, target=0)

RuntimeError: Expected tensor for argument #1 'indices' to have one of the following scalar types: Long, Int; but got torch.cuda.FloatTensor instead (while checking arguments for embedding)